In [2]:
import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F

import numpy as np

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ArcFace(nn.Module):
    def __init__(self, in_features, n_classes, m, s):
        super(ArcFace, self).__init__()
        self.m = m
        self.s = s
        self.n_classes = n_classes
        k = torch.sqrt(torch.tensor(1 / in_features))
        self.weight = nn.Parameter(
            2 * k * torch.rand(n_classes, in_features) - k,
            requires_grad=True
        )

    def forward(self, input, labels=None):
        # normalize inputs, W and perform FC
        x = F.linear(
            F.normalize(input),
            F.normalize(self.weight)
        )
        if labels is None:
            # inference mode
            return x
        # for numerical stability
        x = x.clamp(-1, 1)
        # apply arccos
        x = torch.acos(x)
        # get indexes for labels
        idxes = F.one_hot(labels, self.n_classes) == 1
        # add margin
        x[idxes] += self.m
        # return back to cos
        x = torch.cos(x)
        # rescale
        x = x * self.s
        return x

**ArcFace**

In [33]:
batch_size = 128
num_epochs = 5
device = torch.device('cpu')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout()
        )
        m = 0.4
        s = 2
        self.arcface_layer = ArcFace(50, 10, m, s)

    def forward(self, x, labels=None):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        x = self.arcface_layer(x, labels)
        return x

In [34]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, target)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 3.072172
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.586266
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.276436
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.102610
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.061810

Test set: Average loss: 0.0126, Accuracy: 9263/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.996693
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.981841
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.954064
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.937971
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.936182

Test set: Average loss: 0.0121, Accuracy: 9530/10000 (95%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.921602
Train Epoch: 3 [12800/60000 (21%)]	Loss: 1.845205
Train Epoch: 3 [25600/60000 (43%)]	Loss: 1.816983
Train Epoch: 3 [38400/60000 (64%)]	Loss: 1.836913
Train Epoch: 3 [51200/60000 (85%)]	Loss: 1.880275

Test set: Average loss: 0.0118, Accuracy: 9656/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.861480
Train Epoch: 4 [12800/60000 (21%)]	Lo

**Default FC+SM+CE**

In [31]:
batch_size = 128
num_epochs = 5
device = torch.device('cpu')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x

In [32]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318436
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.232664
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.599157
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.124471
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.963092

Test set: Average loss: 0.0052, Accuracy: 8803/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.790134
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.780597
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.559510
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.528535
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.669486

Test set: Average loss: 0.0027, Accuracy: 9310/10000 (93%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.512292
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.427146
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.351245
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.592325
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.511728

Test set: Average loss: 0.0019, Accuracy: 9471/10000 (95%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.324899
Train Epoch: 4 [12800/60000 (21%)]	Lo